In [0]:
%run ./Child2

In [0]:
import time, json, traceback
import mlflow
from pathlib import Path
from mlflow.tracking import MlflowClient

In [0]:
MLFLOW_EXPERIMENT = "/Workspace/9900-f18a-cake/classifier"
PLANS = [
    {
        "node_id": "ZERO2",
        "only_node": "Haematological malignancy",   # or leave None to let the trainer iterate all children
        "disable_dm": True,             # keep DM off to rely on prefilter top-k
        "prefilter_topk": 200,
        "prefilter_scan_max": 20000,
        "prefilter_chunk_size": 5000,
        "rf_n_jobs": 1,
        "cv_n_jobs": 1,
        "mlflow_experiment_path": MLFLOW_EXPERIMENT,
    },
]

In [0]:
results = []
print(f"Import OK. Will run {len(PLANS)} child job(s).")

In [0]:
print("tracking_uri:", mlflow.get_tracking_uri())
print("MLFLOW_EXPERIMENT:", MLFLOW_EXPERIMENT)

In [0]:
for i, plan in enumerate(PLANS, 1):
    node_id = plan.get("node_id", f"node{i}")
    print(f"\n==== Running child {i}/{len(PLANS)}: node_id={node_id} ====")
    try:
        child_out = run_child(plan)  # run the function defined in Child2
        print("Child OUT:", json.dumps(child_out, indent=2))
        results.append({"ok": True, "node_id": node_id, "result": child_out})
    except Exception as e:
        print("Child FAILED:", e)
        results.append({
            "ok": False,
            "node_id": node_id,
            "error": f"{type(e).__name__}: {e}",
            "trace": traceback.format_exc(),
        })

In [0]:
mlflow.set_experiment(MLFLOW_EXPERIMENT)
with mlflow.start_run(run_name="orchestrator") as run:
    out_path = Path("/dbfs/tmp/mt_method2_parent/child_results.json")
    out_path.parent.mkdir(parents=True, exist_ok=True)
    out_path.write_text(json.dumps(results, indent=2), encoding="utf-8")
    mlflow.log_artifact(str(out_path), artifact_path="orchestration")

print("\nAll done.")